In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'D:\Project\models')
mixer.init()
sound = mixer.Sound(r'D:\Project\alarm.wav')
cap = cv2.VideoCapture(0)
closed_threshold = 0.40
open_threshold = 0.80
score_frames = 10
alarm_frames = 20  # Number of consecutive frames to trigger the alarm
is_alarm_on = False

while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=1)
    
    cv2.rectangle(frame, (0, height - 50), (200, height), (0, 0, 0), thickness=cv2.FILLED)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(255, 0, 0), thickness=3)
        
    for (ex, ey, ew, eh) in eyes:
        eye = frame[ey:ey+eh, ex:ex+ew]
        eye = cv2.resize(eye, (80, 80))
        eye = eye / 255
        eye = eye.reshape(80, 80, 3)
        eye = np.expand_dims(eye, axis=0)
        prediction = model.predict(eye)
        
        if prediction[0][0] > closed_threshold:
            cv2.putText(frame, 'closed', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score' + str(score_frames), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
            score_frames += 1
            if score_frames >= alarm_frames and not is_alarm_on:
                try:
                    sound.play()
                    is_alarm_on = True
                except:
                    pass
            
        elif prediction[0][1] > open_threshold:
            cv2.putText(frame, 'open', (10, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score' + str(score_frames), (100, height - 20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale=1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
            score_frames = max(0, score_frames - 1)  # Ensure score_frames doesn't go below zero
            if score_frames < alarm_frames and is_alarm_on:
                try:
                    sound.stop()
                    is_alarm_on = False
                except:
                    pass
            
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


pygame 2.2.0 (SDL 2.0.22, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 36ms/step
